In [2]:
import ptsFromCAD as pfc
import meshingComponents as mc
import geometricClasses as gcl
import loadModel as lm

import pyfe3d.beamprop as pbp
import pyfe3d.shellprop_utils as psp
import pyfe3d as pf3

import pyfe3Dgcl as p3g

import numpy as np
import numpy.typing as nt
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import scipy.sparse.linalg as ssl
import copy

In this file we will take single components from meshing components and simulate them separately to see what happens. We can say that all the sprint/ptmass eles are fine, so we only need to look at spars and panels.

In [ ]:
#constants/givens
BAT_MASS_1WING = 17480 #kg
E = 72e9 #Pa
NU = .33 #[-]
RHO = 2700 #kg/m^3
RHO_STEEL = 7850 #kg/m^3 #TODO: verify
E_STEEL = 200e9 #Pa #TODO: verif
FULLSPAN = 21 #m
G0 = 9.81 #N/kg
MTOM = 76000 #kg
INFTY_STIFF = 1e20

data = "5000;0;-6.5|4750;0;-24|4500;0;-41|4000;0;-75|3500;0;-107|3000;0;-138|2500;0;-167|2000;0;-190|1500;0;-206|1250;0;-211|1000;0;-211.5|750;0;-205|500;0;-187.5|375;0;-173|250;0;-150.5|125;0;-113.5|62.5;0;-82.5|0;0;0|62.5;0;107.5|125;0;149.5|250;0;206.5|375;0;248|500;0;281.5|750;0;330.5|1000;0;363|1250;0;383.5|1500;0;394|2000;0;390|2500;0;362|3000;0;318|3500;0;259|4000;0;187.5|4500;0;104|4750;0;57|5000;0;6.5&4250;18000;2206.872|4125;18000;2198.122|4000;18000;2189.622|3750;18000;2172.622|3500;18000;2156.622|3250;18000;2141.122|3000;18000;2126.622|2750;18000;2115.122|2500;18000;2107.122|2375;18000;2104.622|2250;18000;2104.372|2125;18000;2107.622|2000;18000;2116.372|1937.5;18000;2123.622|1875;18000;2134.872|1812.5;18000;2153.372|1781.25;18000;2168.872|1750;18000;2210.122|1781.25;18000;2263.872|1812.5;18000;2284.872|1875;18000;2313.372|1937.5;18000;2334.122|2000;18000;2350.872|2125;18000;2375.372|2250;18000;2391.622|2375;18000;2401.872|2500;18000;2407.122|2750;18000;2405.122|3000;18000;2391.122|3250;18000;2369.122|3500;18000;2339.622|3750;18000;2303.872|4000;18000;2262.122|4125;18000;2238.622|4250;18000;2213.372&-471.576;3673.46;441.249|126.713;7770.33;945.522|725.002;11867.2;1449.796|1323.292;15964.069;1954.07&3012.266;18950.167;2324.854|4861.588;5721.895;702.56&500;0;0|1999.77;18000;2214.157&3500;0;0|3499.77;18000;2214.157&787.633;1600;66.192|2883.375;1600;66.192|2617.418;1600;526.843|1053.59;1600;526.843&2024.321;18000;2152.592|3273.646;18000;2152.592|3148.758;18000;2368.903|2149.209;18000;2368.903" 
up = pfc.UnpackedPoints(data)
mesh = gcl.Mesh3D()

#mesh settings
nbCoeff = 1
na = 7
nf2 = 3
ntrig = 2
nipCoeff = 1

#geometry settings
dz = .015 #inwards z offset of battery rail
din = .010 #inner diameter of (threaded) battery rail
cspacing = .25 #chordwise panel rib spacing
bspacing = 1.33 #spanwise panel rib spacing
ribflange = 0.0125 #rib flange length, excluding bends at corners
motormass = 1000
lgmass = 5000
hingemass = 500
lemass = 1000
temass = 1000
# motmountwidth = 0.7 #width of the motor mount along y

#loads
n = 0.3 #[-], load factor
FT = 5000 #N, per motor
LD = 20 #[-], lift-drag ratio
nl = 2.

#eleids
spar = "sp"
battery = "bt"
batteryRail = "br"
# batteryMount = "bm"
# railMount = "rm"
panelPlate = "pl"
panelFlange = "fl"
panelRib = "rb"
skin = "sk"
motor = "mo"
lg ="lg"
hinge ="hn"
mount = "mm"

#geometry loading
nb = int(np.ceil(nbCoeff*4*((up.tfb.y-up.ffb.y)/bspacing+1)))

sparsh, sparigrd, sparSecIdxs, a1, a2, f2 = mc.trigspars(mesh, nb, na, nf2, ntrig, spar, up.ffb, up.frb, up.frt, up.fft, up.tfb, up.trb, up.trt, up.tft)

nip = int(np.ceil(nipCoeff*(4*((a1+2*f2)/cspacing+1)))) #obtaining nip so that we get the minimal required amount of elements

#element definitions
#1) thicknesses
t_spar = 0.005 #thickness of spars and their flanges
t_skin = 0.003 #outer skin thickness
t_plate = 0.0025 #panel plate thickness
t_rib = 0.002 #panel rib thickness

# #2) springs
# ks_rivet = p3g.SpringProp(1e5, 1e7, 1e7, 1e5, 1e5, 1e5, 0, 0, 1, 0, 1, 1, .01) #rivets are all oriented along z axis
# # ks_railmount = (1e7, 1e7, 1e7, 1e7, 1e7, 1e7) #rail to flange mount also oriented along z
# # ks_batmount = (1e6, 1e6, 1e6, 1e6, 1e6, 1e6) #battery to rail mount also oriented along z
# ks_motmount =  p3g.SpringProp(INFTY_STIFF, INFTY_STIFF, INFTY_STIFF, INFTY_STIFF, INFTY_STIFF, INFTY_STIFF) #TODO: here orientation will be "fun"

# #3) beams
# prop_rail = p3g.OrientedBeamProp(1, 0, 0)
# prop_rail.A = np.pi/4*din**2
# prop_rail.Iyy = np.pi/64*din**4
# prop_rail.Izz = prop_rail.Iyy
# prop_rail.J = prop_rail.Iyy+prop_rail.Izz
# scf = 5/6 #ASSUMPTION, TODO: verify
# prop_rail.G = scf*E_STEEL/2/(1+0.3)
# prop_rail.intrho = RHO_STEEL*prop_rail.A
# prop_rail.intrhoy2 = RHO_STEEL*prop_rail.Izz
# prop_rail.intrhoz2 = RHO_STEEL*prop_rail.Iyy

#template for the flange prop
prop_flange = p3g.OrientedBeamProp(0, 0, 0)
prop_flange.A = t_rib*ribflange
prop_flange.Izz = t_rib*ribflange**3/12
prop_flange.Iyy = ribflange*t_rib**3/12
scf = 5/6 #ASSUMPTION, TODO: verify
prop_flange.G = scf*E/2/(1+0.3)
prop_flange.intrho = RHO*prop_flange.A
prop_flange.intrhoy2 = RHO*prop_flange.Izz
prop_flange.intrhoz2 = RHO*prop_flange.Iyy

def prop_flange_or(dir): #oriented flange prop
    pfor = copy.deepcopy(prop_flange)
    pfor.xyex = dir[0]
    pfor.xyey = dir[1]
    pfor.xyez = dir[2]
    return pfor

#element dictionary
eleProps = {"quad":{spar:psp.isotropic_plate(thickness=t_spar, E=E, nu=NU, rho=RHO, calc_scf=True), 
                    skin:psp.isotropic_plate(thickness=t_skin, E=E, nu=NU, rho=RHO, calc_scf=True),
                    panelPlate:psp.isotropic_plate(thickness=t_plate, E=E, nu=NU, rho=RHO, calc_scf=True), 
                    panelRib:psp.isotropic_plate(thickness=t_rib, E=E, nu=NU, rho=RHO, calc_scf=True)},
            "spring":{},
            "beam":{panelFlange:prop_flange_or}, 
            "mass":{}}

#exporting mesh to pyfe3d
KC0, M, N, x, y, z, outdict = p3g.eles_from_gcl(mesh, eleProps)

Apply aerodynamic load on what you have

In [4]:
#boundary conditions = fix at y of fuselage boundary
bk = np.zeros(N, dtype=bool)
check = np.isclose(y, up.ffb.y)
for i in range(pf3.DOF):
    bk[i::pf3.DOF] = check
bu = ~bk

sparshNoFl = sparsh[:, sparSecIdxs[1]:sparSecIdxs[-2]+1]
xmesh = np.zeros(sparshNoFl.shape)
ymesh = np.zeros(sparshNoFl.shape)
for i in range(sparshNoFl.shape[0]):
    for j in range(sparshNoFl.shape[1]):
        xmesh[i,j]=sparshNoFl[i,j].x
        ymesh[i,j]=sparshNoFl[i,j].y

f = np.zeros(N)
Ltot = n*G0*MTOM
L, Mx, My = lm.apply_on_wingbox(xmesh.T, ymesh.T, (0,1), (0,1))
f[2::pf3.DOF][sparigrd[:, sparSecIdxs[1]:sparSecIdxs[-2]+1].flatten()] = Ltot*L.T.flatten()
f[3::pf3.DOF][sparigrd[:, sparSecIdxs[1]:sparSecIdxs[-2]+1].flatten()] = Ltot*Mx.T.flatten()
f[4::pf3.DOF][sparigrd[:, sparSecIdxs[1]:sparSecIdxs[-2]+1].flatten()] = Ltot*My.T.flatten()

#applying weight
weight = p3g.weight(M, n*G0, N, pf3.DOF, gcl.Direction3D(0,0,-1))
f+=weight

Solution - We shall see

In [5]:
#checks and solution
KC0uu = KC0[bu, :][:, bu]
fu = f[bu]
uu = ssl.spsolve(KC0uu, fu)
u = np.zeros(N)
u[bu] = uu

w = u[2::pf3.DOF]
v = u[0::pf3.DOF]

# Pyvista

In [16]:
import pyvista
pyvista.set_plot_theme('document')
pyvista.set_jupyter_backend('trame')
pyvista.global_theme.window_size = [600, 400]
pyvista.global_theme.axes.show = True
pyvista.global_theme.anti_aliasing = 'fxaa'
pyvista.global_theme.show_scalar_bar = True

In [7]:
import pyvista as pv
#nodes is our ncoords
elements = outdict["elements"]
nid_pos = outdict["nid_pos"]

'''handling each element separately at each elemnt type gets a diff colour'''
#springs
edges_s = list()
for s in elements["spring"]:
    edges_s.append([nid_pos[s.n1], nid_pos[s.n2]])
colors_s = [1]*len(edges_s)
#beams
edges_b = list()
for s in elements["spring"]:
    edges_b.append([nid_pos[s.n1], nid_pos[s.n2]])
colors_b = [2]*len(edges_s)
#quads
edges_q = list()
for s in elements["quad"]:
    edges_q.append([nid_pos[s.n1], nid_pos[s.n2]])
    edges_q.append([nid_pos[s.n2], nid_pos[s.n3]])
    edges_q.append([nid_pos[s.n3], nid_pos[s.n4]])
    edges_q.append([nid_pos[s.n4], nid_pos[s.n1]])
colors_q = [3]*len(edges_q)

edges = np.array(edges_s+edges_b+edges_q)
colors = np.array(colors_s+colors_b+colors_q)

# We must "pad" the edges to indicate to vtk how many points per edge
padding = np.empty(edges.shape[0], int) * 2
padding[:] = 2
edges_w_padding = np.vstack((padding, edges.T)).T

mesh_ = pyvista.PolyData(outdict["ncoords"], edges_w_padding)

mesh_.plot(
    scalars=colors,
    render_lines_as_tubes=True,
    style='wireframe',
    line_width=1,
    cmap='jet',
    show_scalar_bar=False,
    background='w',
)

Widget(value='<iframe src="http://localhost:61249/index.html?ui=P_0x242fb1fa330_0&reconnect=auto" class="pyvis…

defmesh

In [9]:
#nodes is our ncoords
elements = outdict["elements"]
nid_pos = outdict["nid_pos"]
nodex = x+v
nodey = y+u[1::pf3.DOF]
nodez = z+w
nodes = list(zip(nodex, nodey, nodez))
miny, maxy = 0,20

'''handling each element separately at each elemnt type gets a diff colour'''
#springs
edges_s = list()
for s in elements["spring"]:
    if y[nid_pos[s.n1]]>miny and y[nid_pos[s.n1]]<maxy:
        edges_s.append([nid_pos[s.n1], nid_pos[s.n2]])
colors_s = [7]*len(edges_s)
#beams
edges_b = list()
for s in elements["spring"]:
    if y[nid_pos[s.n1]]>miny and y[nid_pos[s.n1]]<maxy:
        edges_b.append([nid_pos[s.n1], nid_pos[s.n2]])
colors_b = [5]*len(edges_s)
#quads
edges_q = list()
for s in elements["quad"]:
    if y[nid_pos[s.n1]]>miny and y[nid_pos[s.n1]]<maxy:
        edges_q.append([nid_pos[s.n1], nid_pos[s.n2]])
        edges_q.append([nid_pos[s.n2], nid_pos[s.n3]])
        edges_q.append([nid_pos[s.n3], nid_pos[s.n4]])
        edges_q.append([nid_pos[s.n4], nid_pos[s.n1]])
colors_q = [0]*len(edges_q)

edges = np.array(edges_s+edges_b+edges_q)
colors = np.array(colors_s+colors_b+colors_q)

# We must "pad" the edges to indicate to vtk how many points per edge
padding = np.empty(edges.shape[0], int) * 2
padding[:] = 2
edges_w_padding = np.vstack((padding, edges.T)).T

mesh_ = pyvista.PolyData(nodes, edges_w_padding)

mesh_.plot(
    scalars=colors,
    render_lines_as_tubes=True,
    style='wireframe',
    line_width=1,
    cmap='jet',
    show_scalar_bar=False,
    background='w',
)

Widget(value='<iframe src="http://localhost:61249/index.html?ui=P_0x242fb71af90_2&reconnect=auto" class="pyvis…

Smooth torsion, all good. Now, we plot only panel+spars.

In [4]:
sparBotConns = sparSecIdxs[0::6]
sparTopConns = sparSecIdxs[3::6]
sbci = [sparigrd[:,idx] for idx in sparBotConns] #spar bottom connection ids
stci = [sparigrd[:,idx] for idx in sparTopConns] #spar top connection ids

In [9]:
topflsh, topsksh, topflids, topskids = mc.panel(mesh, up.fft, up.frt, up.tft, up.trt, nb, nip, nf2, 
                                    panelPlate, skin, panelRib, panelFlange, up.surft, stci, cspacing, bspacing)

In [12]:
#exporting mesh to pyfe3d
KC0, M, N, x, y, z, outdict = p3g.eles_from_gcl(mesh, eleProps)

In [13]:
#boundary conditions = fix at y of fuselage boundary
n=1.5
bk = np.zeros(N, dtype=bool)
check = np.isclose(y, up.ffb.y)
for i in range(pf3.DOF):
    bk[i::pf3.DOF] = check
bu = ~bk

xmesh = np.zeros(topsksh.shape)
ymesh = np.zeros(topsksh.shape)
for i in range(topsksh.shape[0]):
    for j in range(topsksh.shape[1]):
        xmesh[i,j]=topsksh[i,j].x
        ymesh[i,j]=topsksh[i,j].y

f = np.zeros(N)
Ltot = n*G0*MTOM
L, Mx, My = lm.apply_on_wingbox(xmesh.T, ymesh.T, (up.fft.y/FULLSPAN, up.tft.y/FULLSPAN), (up.xcft, up.xcrt))
f[2::pf3.DOF][topskids.flatten()] = Ltot*L.T.flatten()
f[3::pf3.DOF][topskids.flatten()] = Ltot*Mx.T.flatten()
f[4::pf3.DOF][topskids.flatten()] = Ltot*My.T.flatten()

#applying weight
weight = p3g.weight(M, n*G0, N, pf3.DOF, gcl.Direction3D(0,0,-1))
f+=weight

In [14]:
#checks and solution
KC0uu = KC0[bu, :][:, bu]
fu = f[bu]
uu = ssl.spsolve(KC0uu, fu)
u = np.zeros(N)
u[bu] = uu

w = u[2::pf3.DOF]
v = u[0::pf3.DOF]

# Same treatment - Pyvista

In [15]:
import pyvista as pv
#nodes is our ncoords
elements = outdict["elements"]
nid_pos = outdict["nid_pos"]

'''handling each element separately at each elemnt type gets a diff colour'''
#springs
edges_s = list()
for s in elements["spring"]:
    edges_s.append([nid_pos[s.n1], nid_pos[s.n2]])
colors_s = [1]*len(edges_s)
#beams
edges_b = list()
for s in elements["spring"]:
    edges_b.append([nid_pos[s.n1], nid_pos[s.n2]])
colors_b = [2]*len(edges_s)
#quads
edges_q = list()
for s in elements["quad"]:
    edges_q.append([nid_pos[s.n1], nid_pos[s.n2]])
    edges_q.append([nid_pos[s.n2], nid_pos[s.n3]])
    edges_q.append([nid_pos[s.n3], nid_pos[s.n4]])
    edges_q.append([nid_pos[s.n4], nid_pos[s.n1]])
colors_q = [3]*len(edges_q)

edges = np.array(edges_s+edges_b+edges_q)
colors = np.array(colors_s+colors_b+colors_q)

# We must "pad" the edges to indicate to vtk how many points per edge
padding = np.empty(edges.shape[0], int) * 2
padding[:] = 2
edges_w_padding = np.vstack((padding, edges.T)).T

mesh_ = pyvista.PolyData(outdict["ncoords"], edges_w_padding)

mesh_.plot(
    scalars=colors,
    render_lines_as_tubes=True,
    style='wireframe',
    line_width=1,
    cmap='jet',
    show_scalar_bar=False,
    background='w',
)

Widget(value='<iframe src="http://localhost:61249/index.html?ui=P_0x2430a964cb0_3&reconnect=auto" class="pyvis…

In [16]:
#nodes is our ncoords
elements = outdict["elements"]
nid_pos = outdict["nid_pos"]
nodex = x+v
nodey = y+u[1::pf3.DOF]
nodez = z+w
nodes = list(zip(nodex, nodey, nodez))
miny, maxy = 0,20

'''handling each element separately at each elemnt type gets a diff colour'''
#springs
edges_s = list()
for s in elements["spring"]:
    if y[nid_pos[s.n1]]>miny and y[nid_pos[s.n1]]<maxy:
        edges_s.append([nid_pos[s.n1], nid_pos[s.n2]])
colors_s = [7]*len(edges_s)
#beams
edges_b = list()
for s in elements["spring"]:
    if y[nid_pos[s.n1]]>miny and y[nid_pos[s.n1]]<maxy:
        edges_b.append([nid_pos[s.n1], nid_pos[s.n2]])
colors_b = [5]*len(edges_s)
#quads
edges_q = list()
for s in elements["quad"]:
    if y[nid_pos[s.n1]]>miny and y[nid_pos[s.n1]]<maxy:
        edges_q.append([nid_pos[s.n1], nid_pos[s.n2]])
        edges_q.append([nid_pos[s.n2], nid_pos[s.n3]])
        edges_q.append([nid_pos[s.n3], nid_pos[s.n4]])
        edges_q.append([nid_pos[s.n4], nid_pos[s.n1]])
colors_q = [0]*len(edges_q)

edges = np.array(edges_s+edges_b+edges_q)
colors = np.array(colors_s+colors_b+colors_q)

# We must "pad" the edges to indicate to vtk how many points per edge
padding = np.empty(edges.shape[0], int) * 2
padding[:] = 2
edges_w_padding = np.vstack((padding, edges.T)).T

mesh_ = pyvista.PolyData(nodes, edges_w_padding)

mesh_.plot(
    scalars=colors,
    render_lines_as_tubes=True,
    style='wireframe',
    line_width=1,
    cmap='jet',
    show_scalar_bar=False,
    background='w',
)

Widget(value='<iframe src="http://localhost:61249/index.html?ui=P_0x242fb3954c0_4&reconnect=auto" class="pyvis…

# Ok, it seems fine, at this point idk. Let's add stuff like batteries, might help to see what goes wrong

In [32]:
beamids, batids = [[]]*(2*ntrig+1), [[]]*(2*ntrig+1)
for i, ids in zip(range(1,2*ntrig+1,2), sbci[1:-1]): #bottom batteries
    beamids[i], batids[i] = mc.bat_rail(mesh, ntrig, a1, a2, f2, din, dz, ids, 
                                        batteryRail, battery, mount, mount, BAT_MASS_1WING)
for i, ids in zip(range(0,2*ntrig+1,2), stci): #top batteries
    beamids[i], batids[i] = mc.bat_rail(mesh, ntrig, a1, a2, f2, din, -dz, ids, 
                                        batteryRail, battery, mount, mount, BAT_MASS_1WING)

In [5]:
ks_motmount =  p3g.SpringProp(INFTY_STIFF, INFTY_STIFF, INFTY_STIFF, INFTY_STIFF, INFTY_STIFF, INFTY_STIFF) #TODO: here orientation will be "fun"

#3) beams
prop_rail = p3g.OrientedBeamProp(1, 0, 0)
prop_rail.A = np.pi/4*din**2
prop_rail.Iyy = np.pi/64*din**4
prop_rail.Izz = prop_rail.Iyy
prop_rail.J = prop_rail.Iyy+prop_rail.Izz
scf = 5/6 #ASSUMPTION, TODO: verify
prop_rail.G = scf*E_STEEL/2/(1+0.3)
prop_rail.intrho = RHO_STEEL*prop_rail.A
prop_rail.intrhoy2 = RHO_STEEL*prop_rail.Izz
prop_rail.intrhoz2 = RHO_STEEL*prop_rail.Iyy

#template for the flange prop
prop_flange = p3g.OrientedBeamProp(0, 0, 0)
prop_flange.A = t_rib*ribflange
prop_flange.Izz = t_rib*ribflange**3/12
prop_flange.Iyy = ribflange*t_rib**3/12
scf = 5/6 #ASSUMPTION, TODO: verify
prop_flange.G = scf*E/2/(1+0.3)
prop_flange.intrho = RHO*prop_flange.A
prop_flange.intrhoy2 = RHO*prop_flange.Izz
prop_flange.intrhoz2 = RHO*prop_flange.Iyy

def prop_flange_or(dir): #oriented flange prop
    pfor = copy.deepcopy(prop_flange)
    pfor.xyex = dir[0]
    pfor.xyey = dir[1]
    pfor.xyez = dir[2]
    return pfor

#element dictionary
eleProps = {"quad":{spar:psp.isotropic_plate(thickness=t_spar, E=E, nu=NU, rho=RHO, calc_scf=True), 
                    skin:psp.isotropic_plate(thickness=t_skin, E=E, nu=NU, rho=RHO, calc_scf=True),
                    panelPlate:psp.isotropic_plate(thickness=t_plate, E=E, nu=NU, rho=RHO, calc_scf=True), 
                    panelRib:psp.isotropic_plate(thickness=t_rib, E=E, nu=NU, rho=RHO, calc_scf=True)},
            "spring":{mount:ks_motmount},
            "beam":{batteryRail:prop_rail, panelFlange:prop_flange_or}, 
            "mass":{battery:lambda m:m, motor:motormass, lg:lgmass, hinge:hingemass}}
print(mount)

mm


In [6]:
botflsh, botsksh, botflids, botskids = mc.panel(mesh, up.ffb, up.frb, up.tfb, up.trb, nb, nip, nf2, 
                                    panelPlate, skin, panelRib, panelFlange, up.surfb, sbci, cspacing, bspacing)
    


In [ ]:
mids = mc.motors(mesh, up.motors, topflsh, topflids, botflsh, botflids, motor, mount)


In [ ]:
lgid = mc.lg(mesh, up.lg, botflsh, botflids, lg, mount)


In [ ]:
hingeid = mc.hinge(mesh, up.hinge, topflsh, topflids, hinge, mount)


In [ ]:
leids, teids = mc.LETE(mesh, up.leline, up.teline, topflsh, topflids, botflsh, botflids, lemass, temass, battery, mount)

In [11]:
#exporting mesh to pyfe3d
KC0, M, N, x, y, z, outdict = p3g.eles_from_gcl(mesh, eleProps)

Now, we apply both weight and lift

In [19]:
n=2.5
#boundary conditions = fix at y of fuselage boundary
bk = np.zeros(N, dtype=bool)
check = np.isclose(y, up.ffb.y)
for i in range(pf3.DOF):
    bk[i::pf3.DOF] = check
bu = ~bk

xmesh = np.zeros(topsksh.shape)
ymesh = np.zeros(topsksh.shape)
for i in range(topsksh.shape[0]):
    for j in range(topsksh.shape[1]):
        xmesh[i,j]=topsksh[i,j].x
        ymesh[i,j]=topsksh[i,j].y

f = np.zeros(N)
Ltot = n*G0*MTOM
L, Mx, My = lm.apply_on_wingbox(xmesh.T, ymesh.T, (up.fft.y/FULLSPAN, up.tft.y/FULLSPAN), (up.xcft, up.xcrt))
f[2::pf3.DOF][topskids.flatten()] = Ltot*L.T.flatten()
f[3::pf3.DOF][topskids.flatten()] = Ltot*Mx.T.flatten()
f[4::pf3.DOF][topskids.flatten()] = Ltot*My.T.flatten()

xmesh = np.zeros(botsksh.shape)
ymesh = np.zeros(botsksh.shape)
for i in range(botsksh.shape[0]):
    for j in range(botsksh.shape[1]):
        xmesh[i,j]=botsksh[i,j].x
        ymesh[i,j]=botsksh[i,j].y

Ltot = n*G0*MTOM
L, Mx, My = lm.apply_on_wingbox(xmesh.T, ymesh.T, (up.ffb.y/FULLSPAN, up.tfb.y/FULLSPAN), (up.xcfb, up.xcrb), False)
f[2::pf3.DOF][botskids.flatten()] = Ltot*L.T.flatten()
f[3::pf3.DOF][botskids.flatten()] = Ltot*Mx.T.flatten()
f[4::pf3.DOF][botskids.flatten()] = Ltot*My.T.flatten()

#applying weight
weight = p3g.weight(M, n*G0, N, pf3.DOF, gcl.Direction3D(0,0,-1))
f+=weight

In [ ]:
#checks and solution
KC0uu = KC0[bu, :][:, bu]
fu = f[bu]
uu = ssl.spsolve(KC0uu, fu)
u = np.zeros(N)
u[bu] = uu

w = u[2::pf3.DOF]
v = u[0::pf3.DOF]

## Pyvista n.o 3

In [17]:
import pyvista as pv
#nodes is our ncoords
elements = outdict["elements"]
nid_pos = outdict["nid_pos"]

'''handling each element separately at each elemnt type gets a diff colour'''
#springs
edges_s = list()
for s in elements["spring"]:
    edges_s.append([nid_pos[s.n1], nid_pos[s.n2]])
colors_s = [1]*len(edges_s)
#beams
edges_b = list()
for s in elements["spring"]:
    edges_b.append([nid_pos[s.n1], nid_pos[s.n2]])
colors_b = [2]*len(edges_s)
#quads
edges_q = list()
for s in elements["quad"]:
    edges_q.append([nid_pos[s.n1], nid_pos[s.n2]])
    edges_q.append([nid_pos[s.n2], nid_pos[s.n3]])
    edges_q.append([nid_pos[s.n3], nid_pos[s.n4]])
    edges_q.append([nid_pos[s.n4], nid_pos[s.n1]])
colors_q = [3]*len(edges_q)

edges = np.array(edges_s+edges_b+edges_q)
colors = np.array(colors_s+colors_b+colors_q)

# We must "pad" the edges to indicate to vtk how many points per edge
padding = np.empty(edges.shape[0], int) * 2
padding[:] = 2
edges_w_padding = np.vstack((padding, edges.T)).T

mesh_ = pyvista.PolyData(outdict["ncoords"], edges_w_padding)

mesh_.plot(
    scalars=colors,
    render_lines_as_tubes=True,
    style='wireframe',
    line_width=1,
    cmap='jet',
    show_scalar_bar=False,
    background='w',
)

Widget(value='<iframe src="http://localhost:50297/index.html?ui=P_0x23c2e8279e0_0&reconnect=auto" class="pyvis…

In [18]:
#nodes is our ncoords
elements = outdict["elements"]
nid_pos = outdict["nid_pos"]
nodex = x+v
nodey = y+u[1::pf3.DOF]
nodez = z+w
nodes = list(zip(nodex, nodey, nodez))
miny, maxy = 0,20

'''handling each element separately at each elemnt type gets a diff colour'''
#springs
edges_s = list()
for s in elements["spring"]:
    if y[nid_pos[s.n1]]>miny and y[nid_pos[s.n1]]<maxy:
        edges_s.append([nid_pos[s.n1], nid_pos[s.n2]])
colors_s = [7]*len(edges_s)
#beams
edges_b = list()
for s in elements["spring"]:
    if y[nid_pos[s.n1]]>miny and y[nid_pos[s.n1]]<maxy:
        edges_b.append([nid_pos[s.n1], nid_pos[s.n2]])
colors_b = [5]*len(edges_s)
#quads
edges_q = list()
for s in elements["quad"]:
    if y[nid_pos[s.n1]]>miny and y[nid_pos[s.n1]]<maxy:
        edges_q.append([nid_pos[s.n1], nid_pos[s.n2]])
        edges_q.append([nid_pos[s.n2], nid_pos[s.n3]])
        edges_q.append([nid_pos[s.n3], nid_pos[s.n4]])
        edges_q.append([nid_pos[s.n4], nid_pos[s.n1]])
colors_q = [0]*len(edges_q)

edges = np.array(edges_s+edges_b+edges_q)
colors = np.array(colors_s+colors_b+colors_q)

# We must "pad" the edges to indicate to vtk how many points per edge
padding = np.empty(edges.shape[0], int) * 2
padding[:] = 2
edges_w_padding = np.vstack((padding, edges.T)).T

mesh_ = pyvista.PolyData(nodes, edges_w_padding)

mesh_.plot(
    scalars=colors,
    render_lines_as_tubes=True,
    style='wireframe',
    line_width=1,
    cmap='jet',
    show_scalar_bar=False,
    background='w',
)

Widget(value='<iframe src="http://localhost:50297/index.html?ui=P_0x23c2fff6a50_1&reconnect=auto" class="pyvis…